<a href="https://colab.research.google.com/github/kevinmenz/Solucao_Reforma_Facil_GEMINI_AI/blob/main/ChatBot_Reforma_F%C3%A1cil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objetivo do Projeto**

Criação de um chatbot com IA que apoie qualquer usuário interessado em reformar seu imóvel.

# **Premissas do projeto**

* 1º Compreender a jornada de uma reforma de imóvel.
* 2º Listar os pontos relevantes da jornada a serem considerados no projeto.
* 3º Elaborar um fluxo dos pontos importantes estruturando as variáveis que serão coletadas para informe posterior da IA.
* 4º criação de código Python que efetive a coleta das informações, repasse formatado seguindo as técnicas de engenharia de prompt da Imersão AI e obtenha uma proposta de projeto com orçamento estimado ao usuário final.
* 5º Resultado obtido aplicando as técnicas da Imersão AI ALURA 2024

# **Tópico 1 - Itens da Jornada**
Utilizando o google Gemini AI Studio efetivamos o respectivo prompt "Quais os pontos mais importantes em uma reforma de imóvel?"

Como resposta obtemos
* Planejamento / Escopo
* Orçamento
* Execução
* Acabamentos
* Inspeção e Entrega
* Outros Pontos Importantes:
  * Comunicar-se claramente com o empreiteiro
  * Manter um registro de todas as despesas e contratos
  * Considerar o valor de revenda e o impacto da reforma no mercado imobiliário local.


# **Tópico 2 - Pontos relevantes s Jornada a serem usados no projeto**

Para o nosso projeto focaremos nos itens planejamento/escopo com o objetivo de gerar uma proposta orçamentário dentro do cenário de cada usuario/cliente que interagir com o bot.

Alguns dos pontos relevantes para o planejamento e definição do escopo são:

* 1º Se a reforma é de um imóvel comercial ou residencial
* 2º Quantidade de cômodos a serem reformados e suas caracteristicas como dimensão, elétrica etc.
* 3º Localização do imóvel (Estado e Cidade)
* 4º Tipo de reforma, precisamos compreender o que precisa ser feito em cada cômodo para elaborar o orçamento da maneira mais precisa possível.





# 3 Comparativo fluxo de construção chatbot abordagem tradicional(if-else-for) versus abordagem Gemini AI (IA generativa)



In [4]:
 !pip install matplotlib

In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('Fluxo Reforma Fácil.drawio.png')
plt.figure(figsize=(200, 600))
plt.imshow(img)
plt.axis('off')
plt.show()


# 4 - Criação do código do chatbot

In [1]:
# instalação do SDK Gemini AI
!pip install -q -U google-generativeai
!pip install --upgrade google-cloud-aiplatform # Necessário se for utilizar a Vertex AI API


In [4]:
import google.generativeai as genai
from google.colab import userdata
import vertexai # Necessário se for utilizar a Vertex AI API
from vertexai.generative_models import GenerativeModel, ChatSession # Necessário se for utilizar a Vertex AI API

# # Definindo o ID do Projeto # Necessário se for utilizar a Vertex AI API
project_id = userdata.get('ID_PROJETO') # altere o valor dessa variável pelo JD do porjeto Vertex

# # Definindo a localização do projeto # Necessário se for utilizar a Vertex AI API
location = userdata.get('LOCALIDADE') # altere o valor dessa variável pelo local do seu porjeto Vertex
# # Iniciatilizando a API Vertex AI # Necessário se for utilizar a Vertex AI API
vertexai.init(project=project_id, location=location)

#Definindo a API KEY
GOOGLE_API_KEY = userdata.get('CHAVE_API') #Altere o valor dessa variável por sua chave de API
#Configurando a chave de API na google AI cloud
genai.configure(api_key=GOOGLE_API_KEY)
# link documentação google modelos https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini?hl=pt-br
# Escolhi o modelo gemini-1.0-pro-002 devido a solução focar apenas em texto e permitir utilizar a opção system instructions que permite contextualizar/instruir o modelo antes da primeira interação com o usuário.
model_name = "gemini-1.5-pro-latest"
#model_name = "models/gemini-1.0-pro-latest"

#Definição das instruções iniciais a serem utilizadas pela IA para responder o usuário
system_instruction = """
Você é um especialista em reforma de imóveis da empresa "Reforma Fácil", poliglota que se adequa a língua do cliente e é sucinto em suas explicações.
No primero contato irá dar as boas vindas, explicar seu objetivo em apoiar na elaboração de um orçamento e coletar as respectivas informações:
1º Dados para contato Nome, Telefone e/ou Email.
2º Tipo de imóvel(Residêncial ou Comercial).
3º Quantidade de cômodos, seu tipo(sala,quartos,cozinha, lavanderia,banheiro,garagem), suas caracteristicas e dimensões em metros (largura, altura,comprimento). Caso ele sinale  um ou alguns cômodos foque nesses informados.
4º Localização do imóvel (País, Estado e Cidade) - utilize para referência do valor médio de mão de obra, materiais na região e moeda do País do cliente.
Valide os dados coletados, principalmente se largura, comprimento e ou altura estão corretos.
A proposta deverá ter formato de lista com valores estimados para os respectivos elementos:
1º mão de obra e lista de materiais necessários com sua respectiva quantidade e valor mínimo, valor médio e valor máximo por item e total geral na moeda do País do cliente.
2º Prazo mínimo, médio e máximo em dias para conclusão.
3º Quantidade de pedreiros estimado para o prazo minimo, médio e máximo (considere que quanto mais pedreiros mais rápido, porém maior valor de mão de obra)
Ao final da proposta você precisa:
1º Adicionar orientações adicionais de segurança e validação entrega de acordo com o tipo de imóvel a ser reformado.
2º Perguntar se a proposta atende a necessidade do cliente ou se precisa de algum ajuste. se o Cliente validar a proposta solicite uma nota de 0(ruim) a 10(excelente) do atendimento e um comentário do que mais gostou ou não da experiência.
3º Para os itens com menos de 70% de certeza na estimativa, informe quais informações adicionais ajudaria a melhorar a estimativa da proposta.
Lembre-se de que para calcular o área ou m² piso ou teto é necessário multiplicar a largura x comprimento. Para paredes considere (2x(alturaxcomprimento) + 2x(alturaxlargura)). Não efetive cálculos para medidas que não foram informadas.
"""

generation_config = {
    "candidate_count":1,
    "temperature":0.5,

}

#Definição dos controles de segurança - link referência:https://ai.google.dev/gemini-api/docs/safety-settings?hl=pt-br
safety_setting = {
    "HARASSMENT":"BLOCK_MEDIUM_AND_ABOVE",
    "HATE": "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE"
}

#Inicialização do modelo desejado "models/gemini-1.5-pro-latest"
model = genai.GenerativeModel(model_name=model_name,system_instruction=system_instruction,safety_settings=safety_setting,generation_config=generation_config);
for m in genai.list_models():
  print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004
models/aqa


# 5º Resultado obtido aplicando as técnicas da Imersão AI ALURA 2024

In [5]:
# Utilizando o classe chatSession do Vertex AI API para manter o contexto conforme as diverças interações são realizadas #
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    response = chat.send_message(prompt)
    return response.text

prompt = input("Digite aqui sua mensagem:")
#Teste do fluxo usar o Google
while prompt != "fim":
  response = get_chat_response(chat,prompt)
  print("Resposta:", response, "\n")
  prompt = input("Digite aqui sua mensagem:")

Digite aqui sua mensagem:Oi
Resposta: Olá! Seja bem-vindo(a) à Reforma Fácil. 

Estou aqui para te auxiliar na elaboração de um orçamento para a reforma do seu imóvel. Para começar, preciso coletar algumas informações:

1. **Dados para contato:** Por favor, me informe seu nome completo, telefone e/ou e-mail para que eu possa entrar em contato posteriormente.
2. **Tipo de imóvel:** É um imóvel residencial ou comercial?
3. **Cômodos:** Quantos cômodos tem o imóvel e quais são eles (sala, quartos, cozinha, lavanderia, banheiro, garagem)? Para cada cômodo, me informe as características (por exemplo, se tem piso, azulejo, etc.) e as dimensões em metros (largura, altura e comprimento). 
4. **Localização:** Em qual país, estado e cidade o imóvel está localizado?

Com essas informações, poderei elaborar um orçamento estimado para a sua reforma. 
 

Digite aqui sua mensagem:Legal, meu nome é Kevin, email Kevin.teste@bol.com.br. Meu imóvel é residencial localizado em Ubatuba - SP e gostaria de r